In [1]:
if ((buff = True)==True):
    print("True")

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (3903171048.py, line 1)

In [47]:
data = pd.read_csv("./files/table2.csv").to_html()
data = HTML(data)
display( data )

,Model,Training_Method,CIFAR-10,CIFAR-100,SVHN,train
0,S-CONV,SGD,0%,0%,0%,True
1,S-LOCAL,SGD,0%,0%,0%,True
2,MLP (Neyshabur et al. 2019),SGD,58.1%,NaN,84.3%,False
3,MLP (Mukkamala and Hein 2017),ADAM_RMS,72.2%,39.3%,NaN,False
4,MLP (Mocanu et al. 2018),SET,74.84%,NaN,NaN,False
5,MLP (Urban et al. 2017),deep_conv_teacher,74.3%,NaN,NaN,False
6,MLP (Lin et al. 2016),ZAE,78.62%,NaN,NaN,False
7,3-FC,SGD,0%,0%,0%,True
8,S-FC,SGD,0%,0%,0%,True
9,S-FC,B-lasso(B=0),0%,0%,0%,True


: 

In [29]:
def log_test():
    logger.log_progress("test")

a = [Thread(target=log_test) for i in range(10)]

for i in a :
    print(i.name)
    i.start()
    

Thread-45 (log_test)
Thread-46 (log_test)
Thread-47 (log_test)
Thread-48 (log_test)
Thread-49 (log_test)
Thread-50 (log_test)
Thread-51 (log_test)
Thread-52 (log_test)
Thread-53 (log_test)
Thread-54 (log_test)
testtesttesttesttesttesttesttesttesttest

In [11]:
INIT_PATH = "."
DOWNLOAD_DATA = False

# To use google colab processing uncomment following lines

#INIT_PATH = "MyDrive/inference_and_learning"
#from google.colab import drive
#MOUNT_PATH = "/content/drive"
#drive.mount(MOUNT_PATH)
#INIT_PATH = MOUNT_PATH + "/" + INIT_PATH
#DOWNLOAD_DATA = True

# To train LOCAL change to True
LOCAL_TRAIN = False 
import os.path,sys
sys.path.append(INIT_PATH)

import networks as nnets
import pandas as pd, torch
from typing import Tuple, List
import torchvision
import torch
import json
torch.set_default_dtype(torch.float16)

table2 = pd.read_csv(INIT_PATH+"/files/table2.csv")
table2_net = json.load(open(INIT_PATH+"/files/table2.json"))
RETRAIN = True

if RETRAIN :
    trainer = nnets.ThreadedTrainer(".",INIT_PATH+"/networks_saved/",INIT_PATH + "/files/figure3.json")
    for dataset in ["CIFAR-10","CIFAR-100","SVHN"]:
        for l in range(len(table2)):
            csv_line = table2.iloc[l].to_dict()
            if csv_line["train"]==False or (str(csv_line[dataset]) == "nan"):
                continue
            ID = "_".join([csv_line[i] for i in ["Model","Training_Method"]]+[dataset])
            network = table2_net[ID]
            if network["model"]["name"] in ["S-LOCAL","D-LOCAL"]:
                print(f"Skipping {ID} as it is a localy connected network")
                continue
            
            save_model_0 = False
            if network["model"] == "S-FC" and network["training_method"] == "B-lasso(B=50)":
                save_model_0 = True
            trainer.append(ID,network,save_model_0)
    res = trainer(2)
    for key,item in res.items() :
        table2.loc[table2["Model"] == key.split("_")[0] and table2["Training_Method"] == key.split("_")[1] ,key.split("_")[2]] = item
    table2.to_csv(INIT_PATH+"/files/table2.csv",index=False)
print(table2)

Skipping S-LOCAL_SGD_CIFAR-10 as it is a localy connected network
Skipping S-LOCAL_SGD_CIFAR-100 as it is a localy connected network
Skipping S-LOCAL_SGD_SVHN as it is a localy connected network

Milestone : S-CONV_SGD_CIFAR-10 : Training Start

Milestone : 3-FC_SGD_CIFAR-10 : Training Start


Exception in thread Thread-9 (trainThread):
Traceback (most recent call last):
  File "/home/yuna/anaconda3/envs/cudaenabled/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/yuna/anaconda3/envs/cudaenabled/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yuna/Documents/inference_and_learning/inference_and_learning_final_project/networks/__init__.py", line 270, in trainThread
    key,net,non_zero = self._pd_dict.pop(0)
  File "/home/yuna/Documents/inference_and_learning/inference_and_learning_final_project/networks/__init__.py", line 243, in train
    loss.backward()
  File "/home/yuna/anaconda3/envs/cudaenabled/lib/python3.10/site-packages/torch/_tensor.py", line 488, in backward
    torch.autograd.backward(
  File "/home/yuna/anaconda3/envs/cudaenabled/lib/python3.10/site-packages/torch/autograd/__init__.py", line 197, in backward
    Variable._execution_engine.run_backward(  # Calls int

Finished !
                            Model    Training_Method CIFAR-10 CIFAR-100  \
0                          S-CONV                SGD       0%        0%   
1                         S-LOCAL                SGD       0%        0%   
2     MLP (Neyshabur et al. 2019)                SGD    58.1%       NaN   
3   MLP (Mukkamala and Hein 2017)           ADAM_RMS    72.2%     39.3%   
4        MLP (Mocanu et al. 2018)                SET   74.84%       NaN   
5         MLP (Urban et al. 2017)  deep_conv_teacher    74.3%       NaN   
6           MLP (Lin et al. 2016)                ZAE   78.62%       NaN   
7                            3-FC                SGD       0%        0%   
8                            S-FC                SGD       0%        0%   
9                            S-FC       B-lasso(B=0)       0%        0%   
10                           S-FC      B-lasso(B=10)       0%        0%   
11                           S-FC      B-lasso(B=50)       0%        0%   

     SVHN  tr